In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

import requests
from bs4 import BeautifulSoup
import time


In [2]:
seasons = np.arange(2023, 2024).tolist()
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

In [3]:
def find_stats_link(s, sns, league_code):

    splitted = s.split("/")

    team_name = splitted[4]

    if "fc" in team_name:
        new_team_name = team_name.split("-")[1]+"-"+team_name.split("-")[0]
    else:
        new_team_name = team_name


    return s.replace("/"+team_name, new_team_name).replace('startseite', 'leistungsdaten').split('saison_id')[0] + f"plus/1?reldata={league_code}%26"+str(sns)

In [7]:
#all teams links
def get_links(league_code, seasons):
    base_link = f"https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league_code}/plus/?saison_id="#2021
    base = "https://www.transfermarkt.co.uk/"
    
    all_urls = []
    for s in (seasons):
        url =f"https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league_code}/plus/?saison_id="+str(s)
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        
        team_links = soup.find_all("table", {"class": "items"})
        links = []

        for row in team_links[0].find("tbody").children:
            if row.name == "tr":
                cells = row.find("td", {"class": "hauptlink no-border-links"})
                new_url = (base+cells.find("a").get("href"))
                tn = cells.find("a").text.split(" FC")[0]
                right_url = find_stats_link(new_url, s, league_code)
                links+= [(tn, right_url)]
        all_urls += (links) 
    return all_urls

In [9]:
team_links =get_links('GB1', [2024])

https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/GB1/plus/?saison_id=2024


In [26]:
#stats from squad
def find_squad_stats(als):
    team = als[0]
    link = als[1]
    season = link[-4:] + "-" + str(int(link[-2:])+1)
    url = link
    print(url)
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    new_season = soup.find('div', {'class': 'inline-select'}).find('optgroup').text.split('\n')[1].split(' ')[1]
    new_season = '20'+new_season.split('/')[0]+'-'+new_season.split('/')[1]
    season = new_season
    print(new_season)
    
    team_links = soup.find_all("table", {"class": "items"})
    links = []
    
    all_p = []
    for row in team_links[0].find("tbody").children:
        player = [season, team]
        if row.name == "tr":
            cells = row.find_all("td")
            for cell in cells:
                if cell.find("img"):
                    #nationality
                    
                    if cell.find("td", {"class": "hauptlink"}):
                        player_link =cell.find("td", {"class": "hauptlink"}).find("span", {"class": "hide-for-small"}).find("a")["href"]#[-1]["href"])
                        #print(player_link)
                        #a = 
                        
                             #cell.find("span", {"class": "hide-for-small"}).find("a")["href"]
                        player_link = "https://www.transfermarkt.co.uk" + player_link.replace("profil", "leistungsdatendetails")
                            #player_resp = requests.get(player_link, headers={'User-Agent': 'not found 404'})
                            #player_soup = BeautifulSoup(player_resp.content, "html.parser")
                            #try:
                            #    height = player_soup.find("span", {"itemprop":"height"}).text[:-1].strip().replace(",", ".")
                            #except:
                            #    height = ' '
                            #print(height)
                        player.append(player_link)
                        player.append(player_link.split('/')[-1])
                    
                    elif "posrela" in cell["class"] and "bilderrahmen-fixed" in cell.find("img")["class"]:
                        #print(cell)
                        #print(cell)
                        player.append("not loaned")
                        natts = [n["title"] for n in cell.find_all("img")]
                        #print(natts)
                        player.append(natts[0])

                    elif cell.find("span"):
                        #print(cell)
                        loan_checker = cell.find("span").find("a")["title"]
                        
                        if "On loan from" in loan_checker:
                            player.append(loan_checker)
                        else:
                            player.append("not loaned")
                        player.append(cell.find("tr").find("a").find("img")["title"])
                elif cell.find("a"):
                    #print(cell.find("a")["title"])
                    player.append(cell.find('a')['title'])
                    
                else:
                    value = cell.text
                    if cell.find('div'):#    
                        if cell.find('div')['class']: 
                            if cell.find('div')['class'][0] == 'rn_nummer':
                                if cell.text == '-':
                                    span = row.find('span', class_='wechsel-kader-wappen hide-for-small')
                                    if span:
                                        a_tag = span.find_next('a')
                                        if a_tag and 'title' in a_tag.attrs:
                                            print(a_tag['title'])
                                            value = '0'
                                    else:
                                        value = cell.text
                                else:
                                    value = cell.text
                    
                    player.append(value)

            

            all_p.append(player)
            
    all_p_df = pd.DataFrame(all_p, columns=["Season", "Team", "#", "Link" , 'PLAYER_ID', "Player Name", "Position", "Age", "Appearences", "Played Games", "Goals", "Assists", "Yellow", "Second Yellow", "Red", "Come From Bench", "Substituted", "PPG", "Minute Played"])
    
    for c in all_p_df.columns.values[7:]:#[7:]:
        all_p_df[c] = all_p_df[c].replace("Not in squad during this season", "0")
        all_p_df[c] = all_p_df[c].replace("Not used during this season", "0")
        all_p_df[c] = all_p_df[c].replace("-", "0")
        
        if all_p_df[c].dtype == object:
            all_p_df[c] = all_p_df[c].str.replace("'", "")
            #all_p_df[c] = all_p_df[c].str.replace(",", ".")
            if c=='Minute Played':
                all_p_df[c] = all_p_df[c].str.replace(".", "")
            all_p_df[c] = all_p_df[c].str.replace("-", "")
            all_p_df[c] = all_p_df[c].str.replace("†", "")
        
        try:
            all_p_df[c] = all_p_df[c].astype(float)
        except:
            all_p_df[c] = all_p_df[c]
    
    all_p_df['TEAM_ID'] = link.split('/')[-3]
    return all_p_df

In [28]:
leagues = ['GB1']#, 'L1', 'PO1', 'FR1', 'IT1', 'ES1', 'TR1', 'NL1', 'BE1']

In [30]:
import time

In [35]:
sp = find_squad_stats(team_links[-7])

https://www.transfermarkt.co.uk/bournemouth-afc/leistungsdaten/verein/989/plus/1?reldata=GB1%262024
2024-25
Joined as a winter arrival from: CA Lanús; date: Jan 7, 2025; fee: €8.00m
Joined as a winter arrival from: D.C. United; date: Jan 3, 2025; fee: €1.50m


In [37]:
sp

,Season,Team,#,Link,PLAYER_ID,Player Name,Position,Age,Appearences,Played Games,Goals,Assists,Yellow,Second Yellow,Red,Come From Bench,Substituted,PPG,Minute Played,TEAM_ID
0,2024-25,AFC Bournemouth,13,https://www.transfermarkt.co.uk/kepa-arrizabal...,192279,Kepa Arrizabalaga,Goalkeeper,30.0,15.0,14.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.71,1260.0,989
1,2024-25,AFC Bournemouth,42,https://www.transfermarkt.co.uk/mark-travers/l...,357658,Mark Travers,Goalkeeper,25.0,20.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.75,360.0,989
2,2024-25,AFC Bournemouth,-,https://www.transfermarkt.co.uk/neto/leistungs...,111819,Neto,Goalkeeper,35.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,180.0,989
3,2024-25,AFC Bournemouth,40,https://www.transfermarkt.co.uk/will-dennis/le...,582078,Will Dennis,Goalkeeper,24.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,989
4,2024-25,AFC Bournemouth,27,https://www.transfermarkt.co.uk/ilya-zabarnyi/...,659089,Ilya Zabarnyi,Centre-Back,22.0,20.0,20.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.65,1800.0,989
5,2024-25,AFC Bournemouth,3,https://www.transfermarkt.co.uk/milos-kerkez/l...,730861,Milos Kerkez,Left-Back,21.0,20.0,20.0,1.0,3.0,2.0,0.0,0.0,0.0,4.0,1.65,1731.0,989
6,2024-25,AFC Bournemouth,5,https://www.transfermarkt.co.uk/marcos-senesi/...,469781,Marcos Senesi,Centre-Back,27.0,12.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,1.55,921.0,989
7,2024-25,AFC Bournemouth,2,https://www.transfermarkt.co.uk/dean-huijsen/l...,890290,Dean Huijsen,Centre-Back,19.0,20.0,14.0,2.0,0.0,3.0,0.0,0.0,5.0,0.0,1.79,880.0,989
8,2024-25,AFC Bournemouth,37,https://www.transfermarkt.co.uk/max-aarons/lei...,471690,Max Aarons,Right-Back,25.0,14.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.67,85.0,989
9,2024-25,AFC Bournemouth,22,https://www.transfermarkt.co.uk/julian-araujo/...,513970,Julián Araujo,Right-Back,23.0,11.0,8.0,0.0,0.0,1.0,0.0,0.0,3.0,5.0,1.13,356.0,989


In [25]:
sp.find('div', {'class': 'inline-select'}).find('optgroup').text.split('\n')[1].split(' ')[1]

'24/25'

In [12]:
import pandas as pd
from tqdm import tqdm
import numpy as np

import requests
from bs4 import BeautifulSoup
import time
import sys

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

def find_stats_link(s, sns, league_code):

    splitted = s.split("/")

    team_name = splitted[4]

    if "fc" in team_name:
        new_team_name = team_name.split("-")[1]+"-"+team_name.split("-")[0]
    else:
        new_team_name = team_name


    return s.replace("/"+team_name, new_team_name).replace('startseite', 'leistungsdaten').split('saison_id')[0] + f"plus/1?reldata={league_code}%26"+str(sns)

#all teams links
def get_links(league_code, seasons):
    base_link = f"https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league_code}/plus/?saison_id="#2021
    base = "https://www.transfermarkt.co.uk/"
    
    all_urls = []
    for s in (seasons):
        url =f"https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league_code}/plus/?saison_id="+str(s)
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        
        team_links = soup.find_all("table", {"class": "items"})
        links = []

        for row in team_links[0].find("tbody").children:
            if row.name == "tr":
                cells = row.find("td", {"class": "hauptlink no-border-links"})
                new_url = (base+cells.find("a").get("href"))
                tn = cells.find("a").text.split(" FC")[0]
                right_url = find_stats_link(new_url, s, league_code)
                links+= [(tn, right_url)]
        all_urls += (links) 
    return all_urls

def parse_link_full(link, how='populate'):
    league_name = link.split('=')[1].split('%')[0]
    link = link.replace(league_name, '')
    if how == 'update':
        link = link.split('?')[0]
    return link

#stats from squad
def find_squad_stats(als, how='populate'):
    team = als[0]
    link = als[1]
    season = link[-4:] + "-" + str(int(link[-2:])+1)
    new_link = parse_link_full(link, how=how)
    url = new_link
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    if how == 'update':
        new_season = soup.find('div', {'class': 'inline-select'}).find('optgroup').text.split('\n')[1].split(' ')[1]
        new_season = '20'+new_season.split('/')[0]+'-'+new_season.split('/')[1]
        season = new_season

        
        #print(new_season, url)
    #time.sleep(30309)
    team_links = soup.find_all("table", {"class": "items"})
    links = []
    
    all_p = []
    for row in team_links[0].find("tbody").children:
        player = [season, team]
        if row.name == "tr":
            cells = row.find_all("td")
            for cell in cells:
                if cell.find("img"):
                    #nationality
                    
                    if cell.find("td", {"class": "hauptlink"}):
                        player_link =cell.find("td", {"class": "hauptlink"}).find("span", {"class": "hide-for-small"}).find("a")["href"]#[-1]["href"])
                        #print(player_link)
                        #a = 
                        
                             #cell.find("span", {"class": "hide-for-small"}).find("a")["href"]
                        player_link = "https://www.transfermarkt.co.uk" + player_link.replace("profil", "leistungsdatendetails")
                            #player_resp = requests.get(player_link, headers={'User-Agent': 'not found 404'})
                            #player_soup = BeautifulSoup(player_resp.content, "html.parser")
                            #try:
                            #    height = player_soup.find("span", {"itemprop":"height"}).text[:-1].strip().replace(",", ".")
                            #except:
                            #    height = ' '
                            #print(height)
                        player.append(player_link)
                        player.append(player_link.split('/')[-1])
                    
                    elif "posrela" in cell["class"] and "bilderrahmen-fixed" in cell.find("img")["class"]:
                        #print(cell)
                        #print(cell)
                        player.append("not loaned")
                        natts = [n["title"] for n in cell.find_all("img")]
                        #print(natts)
                        player.append(natts[0])

                    elif cell.find("span"):
                        #print(cell)
                        loan_checker = cell.find("span").find("a")["title"]
                        
                        if "On loan from" in loan_checker:
                            player.append(loan_checker)
                        else:
                            player.append("not loaned")
                        player.append(cell.find("tr").find("a").find("img")["title"])
                elif cell.find("a"):
                    #print(cell.find("a")["title"])
                    player.append(cell.find('a')['title'])
                    
                else:
                    player.append(cell.text)

            

            all_p.append(player)
            
    all_p_df = pd.DataFrame(all_p, columns=["Season", "Team", "#", "Link" , 'PLAYER_ID', "Player Name", "Position", "Age", "Appearences", "Played Games", "Goals", "Assists", "Yellow", "Second Yellow", "Red", "Come From Bench", "Substituted", "PPG", "Minute Played"])
    
    for c in all_p_df.columns.values[7:]:#[7:]:
        all_p_df[c] = all_p_df[c].replace("Not in squad during this season", "0")
        all_p_df[c] = all_p_df[c].replace("Not used during this season", "0")
        all_p_df[c] = all_p_df[c].replace("-", "0")
        
        if all_p_df[c].dtype == object:
            all_p_df[c] = all_p_df[c].str.replace("'", "")
            #all_p_df[c] = all_p_df[c].str.replace(",", ".")
            if c=='Minute Played':
                all_p_df[c] = all_p_df[c].str.replace(".", "",regex=True)

            all_p_df[c] = all_p_df[c].str.replace("-", "")
            all_p_df[c] = all_p_df[c].str.replace("†", "")
        
        try:
            all_p_df[c] = all_p_df[c].astype(float)
        except:
            all_p_df[c] = all_p_df[c]
    
    all_p_df['TEAM_ID'] = link.split('/')[-3]
    return all_p_df

In [14]:
leagues = ['GB1']
seasons = [2024]
base_df = pd.DataFrame()
mode = 'populate'
for l in leagues:
    print(f"getting {l} data from {seasons} - {mode} mode")
    team_links =get_links(l, seasons)
    tl = []
    for team in tqdm(team_links):
        passed = 0
        while passed == 0:
            try:
                tl.append(find_squad_stats(team, how=mode))
                passed = 1
            except:
                print(f'team {team} error - trying again')
                time.sleep(5)
    all_teams = pd.concat(tl)
    all_teams['League'] = [l]*len(all_teams)

    base_df = pd.concat([base_df, all_teams])


getting GB1 data from [2024] - populate mode
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/GB1/plus/?saison_id=2024


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:14<00:00,  3.72s/it]


In [24]:
team_links

[('Manchester City',
  'https://www.transfermarkt.co.uk/manchester-city/leistungsdaten/verein/281/plus/1?reldata=GB1%262024'),
 ('Arsenal',
  'https://www.transfermarkt.co.uk/arsenal-fc/leistungsdaten/verein/11/plus/1?reldata=GB1%262024'),
 ('Chelsea',
  'https://www.transfermarkt.co.uk/chelsea-fc/leistungsdaten/verein/631/plus/1?reldata=GB1%262024'),
 ('Liverpool',
  'https://www.transfermarkt.co.uk/liverpool-fc/leistungsdaten/verein/31/plus/1?reldata=GB1%262024'),
 ('Manchester United',
  'https://www.transfermarkt.co.uk/manchester-united/leistungsdaten/verein/985/plus/1?reldata=GB1%262024'),
 ('Tottenham Hotspur',
  'https://www.transfermarkt.co.uk/tottenham-hotspur/leistungsdaten/verein/148/plus/1?reldata=GB1%262024'),
 ('Newcastle United',
  'https://www.transfermarkt.co.uk/newcastle-united/leistungsdaten/verein/762/plus/1?reldata=GB1%262024'),
 ('Brighton & Hove Albion',
  'https://www.transfermarkt.co.uk/brighton-amp-hove-albion/leistungsdaten/verein/1237/plus/1?reldata=GB1%2620

In [35]:
base_df[((base_df.Appearences > 10) & (base_df['#']=='-')) | (base_df['#']!='-')]

,Season,Team,#,Link,PLAYER_ID,Player Name,Position,Age,Appearences,Played Games,...,Assists,Yellow,Second Yellow,Red,Come From Bench,Substituted,PPG,Minute Played,TEAM_ID,League
0,2024-25,Manchester City,31,https://www.transfermarkt.co.uk/ederson/leistu...,238223,Ederson,Goalkeeper,31.0,25.0,18.0,...,1.0,3.0,0.0,0.0,0.0,0.0,1.56,,281,GB1
1,2024-25,Manchester City,18,https://www.transfermarkt.co.uk/stefan-ortega/...,85941,Stefan Ortega,Goalkeeper,32.0,28.0,10.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.7,,281,GB1
2,2024-25,Manchester City,33,https://www.transfermarkt.co.uk/scott-carson/l...,14555,Scott Carson,Goalkeeper,39.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,281,GB1
5,2024-25,Manchester City,3,https://www.transfermarkt.co.uk/ruben-dias/lei...,258004,Rúben Dias,Centre-Back,27.0,21.0,19.0,...,0.0,4.0,0.0,0.0,3.0,1.0,1.95,,281,GB1
6,2024-25,Manchester City,24,https://www.transfermarkt.co.uk/josko-gvardiol...,475959,Josko Gvardiol,Centre-Back,22.0,28.0,27.0,...,0.0,1.0,0.0,0.0,4.0,3.0,1.67,,281,GB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2024-25,Ipswich Town,21,https://www.transfermarkt.co.uk/chiedozie-ogbe...,392591,Chiedozie Ogbene,Right Winger,27.0,8.0,6.0,...,0.0,0.0,0.0,0.0,2.0,4.0,0.50,,677,GB1
24,2024-25,Ipswich Town,27,https://www.transfermarkt.co.uk/george-hirst/l...,420808,George Hirst,Centre-Forward,25.0,8.0,8.0,...,1.0,1.0,0.0,0.0,7.0,1.0,0.88,,677,GB1
25,2024-25,Ipswich Town,33,https://www.transfermarkt.co.uk/nathan-broadhe...,344009,Nathan Broadhead,Left Winger,26.0,10.0,5.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.60,,677,GB1
26,2024-25,Ipswich Town,7,https://www.transfermarkt.co.uk/wes-burns/leis...,255731,Wes Burns,Right Winger,30.0,17.0,15.0,...,1.0,1.0,0.0,0.0,5.0,10.0,0.73,,677,GB1


In [36]:
tdf = pd.read_csv('../data/players_infos.csv')

In [41]:
tdf[tdf.Team == 'Leicester City'].tail(28)

Position
Defensive Midfield    5
Goalkeeper            4
Centre-Back           4
Centre-Forward        4
Right Winger          3
Left-Back             2
Right-Back            2
Attacking Midfield    2
Central Midfield      1
Left Winger           1
Name: count, dtype: int64

In [10]:
base_df = pd.DataFrame()
for l in leagues:
    team_links =get_links(l, [2023])
    tl = []
    for team in tqdm(team_links):
        passed = 0
        while passed == 0:
            try:
                tl.append(find_squad_stats(team))
                passed = 1
            except:
                print(f'team {team} error - trying again')
                #time.sleep(5)
        break
    all_teams = pd.concat(tl)
    all_teams['League'] = [l]*len(all_teams)

    base_df = pd.concat([base_df, all_teams])

https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/GB1/plus/?saison_id=2023


  0%|                                                                                                                                   | 0/20 [00:00<?, ?it/s]

https://www.transfermarkt.co.uk/manchester-city/leistungsdaten/verein/281/plus/1?reldata=GB1%262023


  0%|                                                                                                                                   | 0/20 [00:01<?, ?it/s]

2024-25


TypeError: cannot concatenate object of type '<class 'bs4.BeautifulSoup'>'; only Series and DataFrame objs are valid

In [ ]:
base_df

In [33]:
base_df.to_csv('players_infos.csv', index=False)

In [23]:
base_df['Played Games'].value_counts()

Played Games
    5126
Name: count, dtype: int64

In [86]:
all_teams = pd.concat(tl)

In [35]:
all_teams

,Season,Team,#,Link,PLAYER_ID,Player Name,Position,Age,Appearences,Played Games,...,Assists,Yellow,Second Yellow,Red,Come From Bench,Substituted,PPG,Minute Played,TEAM_ID,League
0,202425.0,Club Brugge KV,22.0,https://www.transfermarkt.co.uk/simonmignolet/...,50219.0,Simon Mignolet,Goalkeeper,36.0,10.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.50,900.0,2282,BE1
1,202425.0,Club Brugge KV,29.0,https://www.transfermarkt.co.uk/nordinjackers/...,250543.0,Nordin Jackers,Goalkeeper,27.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2282,BE1
2,202425.0,Club Brugge KV,16.0,https://www.transfermarkt.co.uk/danivandenheuv...,559320.0,Dani van den Heuvel,Goalkeeper,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2282,BE1
3,202425.0,Club Brugge KV,55.0,https://www.transfermarkt.co.uk/maximdecuyper/...,429915.0,Maxim De Cuyper,LeftBack,23.0,10.0,10.0,...,1.0,2.0,0.0,0.0,1.0,2.0,1.50,820.0,2282,BE1
4,202425.0,Club Brugge KV,4.0,https://www.transfermarkt.co.uk/joelordonez/le...,893661.0,Joel Ordóñez,CentreBack,20.0,8.0,6.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.67,532.0,2282,BE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,202425.0,FCV Dender EH,26.0,https://www.transfermarkt.co.uk/ragnaroratmang...,410854.0,Ragnar Oratmangoen,Left Winger,26.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.50,62.0,9010,BE1
25,202425.0,FCV Dender EH,19.0,https://www.transfermarkt.co.uk/aliakman/leist...,555719.0,Ali Akman,CentreForward,22.0,10.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.50,21.0,9010,BE1
26,202425.0,FCV Dender EH,98.0,https://www.transfermarkt.co.uk/jordysoladio/l...,533092.0,Jordy Soladio,CentreForward,26.0,10.0,9.0,...,0.0,1.0,0.0,0.0,5.0,4.0,1.56,351.0,9010,BE1
27,202425.0,FCV Dender EH,17.0,https://www.transfermarkt.co.uk/abdoulayeyahay...,779309.0,Abdoulaye Yahaya,Right Winger,23.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,9010,BE1


In [92]:
#find player main stats page
session = requests.Session()

session.headers.update(headers)

def get_page(l):
    passed = 0
    while passed == 0:
        try:
            page = requests.get(l, headers=headers)
            passed = 1
        except:
            passed = 0
            print("trying to download ", l)
            time.sleep(5)
            
    return page

In [95]:
pl_links = []

for link in tqdm(all_teams.Link.values):
    pl_links.append(get_page(link))

100%|████████████████████████████████████████████████████████████████████████████████| 585/585 [14:30<00:00,  1.49s/it]


In [28]:
new_season = '2024-25'
leagues = ['GB2', 'SER1', 'BRA1', 'AR1N', 'DK1', 'GB1', 'L1', 'PO1', 'FR1', 'IT1', 'ES1', 'TR1', 'NL1', 'BE1', 'TS1']

In [29]:
old_df = pd.read_csv('../data/players_infos.csv')
old_df = old_df[~((old_df.Season == new_season) & (old_df.League.isin(leagues)))]

In [30]:
old_df

,Season,Team,#,Link,PLAYER_ID,Player Name,Position,Age,Appearences,Played Games,...,Assists,Yellow,Second Yellow,Red,Come From Bench,Substituted,PPG,Minute Played,TEAM_ID,League
0,2017-18,Manchester City,31,https://www.transfermarkt.co.uk/ederson/leistu...,238223,Ederson,Goalkeeper,23.0,57.0,45.0,...,0.0,2.0,0.0,0.0,0.0,1.0,2.49,NaN,281,GB1
1,2017-18,Manchester City,54,https://www.transfermarkt.co.uk/angus-gunn/lei...,201574,Angus Gunn,Goalkeeper,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,281,GB1
2,2017-18,Manchester City,-,https://www.transfermarkt.co.uk/joe-hart/leist...,40204,Joe Hart,Goalkeeper,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,281,GB1
3,2017-18,Manchester City,1,https://www.transfermarkt.co.uk/claudio-bravo/...,40423,Claudio Bravo,Goalkeeper,34.0,57.0,13.0,...,1.0,0.0,0.0,0.0,1.0,0.0,2.54,NaN,281,GB1
4,2017-18,Manchester City,5,https://www.transfermarkt.co.uk/john-stones/le...,186590,John Stones,Centre-Back,23.0,37.0,29.0,...,0.0,0.0,0.0,0.0,3.0,5.0,2.55,NaN,281,GB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88506,2023-24,Hvidovre IF,22,https://www.transfermarkt.co.uk/andreas-smed/l...,384802,Andreas Smed,Left Winger,26.0,33.0,29.0,...,1.0,2.0,0.0,0.0,13.0,14.0,0.69,NaN,1894,DK1
88507,2023-24,Hvidovre IF,27,https://www.transfermarkt.co.uk/mathias-andrea...,715759,Mathias Andreasen,Centre-Forward,19.0,26.0,18.0,...,0.0,3.0,0.0,0.0,16.0,2.0,0.67,NaN,1894,DK1
88508,2023-24,Hvidovre IF,24,https://www.transfermarkt.co.uk/simon-makienok...,87162,Simon Makienok,Centre-Forward,32.0,23.0,22.0,...,1.0,1.0,0.0,0.0,12.0,6.0,0.64,NaN,1894,DK1
88509,2023-24,Hvidovre IF,19,https://www.transfermarkt.co.uk/marco-ramkilde...,321860,Marco Ramkilde,Centre-Forward,25.0,12.0,6.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.33,NaN,1894,DK1
